# Lecture 8

## Nathan Kunz

In [37]:
import wooldridge as woo
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import itertools
import scipy.stats as stats

## Organizing Panel Data

A panel data model takes rthe following form:

$$y_{it} = \beta_0 + \beta_{1} x_{1it} + \beta_{2} x_{2it}+...+ \beta_{k} x_{kit} + a_i + u_{it}$$

where $i$ is a subscript denoting an individual and $t$ denotes the time period an observation is taken. The error term is broken into a time invariant component $a_i$ and a time variant component $u_{it}$.

In [38]:
# We have three individuals
i = ["Larry", "Dean", "Sarah"]

# We take observations once a year
t = np.arange(2000, 2010)

# each individual is observed once a year
panel_example = pd.DataFrame(itertools.product(t,i), columns = ["year", "person"])

panel_example.head()

,year,person
0,2000,Larry
1,2000,Dean
2,2000,Sarah
3,2001,Larry
4,2001,Dean


In [39]:
panel_example["outcome"] = np.random.normal(0,1,30) 

In [40]:
panel_example["outcome"] = np.random.normal(0,1,30) 
panel_example.head()

,year,person,outcome
0,2000,Larry,0.050978
1,2000,Dean,0.438253
2,2000,Sarah,-0.481485
3,2001,Larry,0.999110
4,2001,Dean,1.214343


In [41]:
panel_example = panel_example.set_index(["year", "person"])
panel_example.head(10)

outcome
year person          
2000 Larry   0.050978
     Dean    0.438253
     Sarah  -0.481485
2001 Larry   0.999110
     Dean    1.214343
     Sarah   0.260120
2002 Larry  -2.201725
     Dean   -0.556271
     Sarah   0.248650
2003 Larry  -0.716523

This is a balanced panel, where all individuals are observed across all periods. Below is an example of an unbalanced panel, where we are missing an observation for Sarah in 2000 and Larry in 2008.

In [42]:
panel_example.drop([(2008, "Larry"), (2000, "Sarah")])

outcome
year person          
2000 Larry   0.050978
     Dean    0.438253
2001 Larry   0.999110
     Dean    1.214343
     Sarah   0.260120
2002 Larry  -2.201725
     Dean   -0.556271
     Sarah   0.248650
2003 Larry  -0.716523
     Dean    0.492800
     Sarah   0.567025
2004 Larry  -0.457913
     Dean   -2.032209
     Sarah   1.208199
2005 Larry   2.553699
     Dean   -0.397787
     Sarah  -0.437191
2006 Larry  -0.115090
     Dean    1.753850
     Sarah   0.978722
2007 Larry   0.724458
     Dean   -0.572340
     Sarah  -1.286034
2008 Dean   -0.144848
     Sarah   0.295902
2009 Larry  -0.003139
     Dean    0.493091
     Sarah   0.991346

## First Differenced Estimator

In [43]:
# There are two time periods
crime2 = woo.data('crime2')
crime2[["year", "area", "crmrte", "unem"]].head()

,year,area,crmrte,unem
0,82,44.599998,74.657562,8.2
1,87,44.599998,70.117294,3.7
2,82,375.000000,92.934868,8.1
3,87,375.000000,89.972214,5.4
4,82,49.799999,83.611130,9.0


In [44]:
crime2.year.unique()

array([82, 87])

In [45]:
# create a time dummy
crime2['t'] = (crime2.year == 87)*1

In [47]:
crime2

,pop,crimes,unem,officers,pcinc,west,nrtheast,south,year,area,...,clpop,clcrmrte,lpolpc,clpolpc,cllawexp,cunem,clpopden,lcrmrt_1,ccrmrte,t
0,229528.0,17136.0,8.2,326,8532,1,0,0,82,44.599998,...,NaN,NaN,0.350872,NaN,NaN,NaN,NaN,NaN,NaN,0
1,246815.0,17306.0,3.7,321,12155,1,0,0,87,44.599998,...,0.072614,-0.062743,0.262802,-0.088070,0.977952,-4.5,0.072615,4.312912,-4.540268,1
2,814054.0,75654.0,8.1,1621,7551,1,0,0,82,375.000000,...,NaN,NaN,0.688772,NaN,NaN,NaN,NaN,NaN,NaN,0
3,933177.0,83960.0,5.4,1803,11363,1,0,0,87,375.000000,...,0.136568,-0.032398,0.658612,-0.030160,0.200762,-2.7,0.136568,4.531899,-2.962654,1
4,374974.0,31352.0,9.0,633,8343,1,0,0,82,49.799999,...,NaN,NaN,0.523614,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,280370.0,20775.0,6.3,623,9340,0,0,1,87,53.000000,...,0.033710,-0.041335,0.798436,0.020713,0.147894,-0.7,0.033710,4.346731,-3.127060,1
88,268887.0,14537.0,5.5,400,7704,0,0,1,82,255.899994,...,NaN,NaN,0.397173,NaN,NaN,NaN,NaN,NaN,NaN,0
89,340158.0,18703.0,6.3,504,12039,0,0,1,87,255.899994,...,0.235119,0.016868,0.393166,-0.004007,0.107985,0.8,0.235119,3.990161,0.919670,1
90,645231.0,45851.0,13.1,2087,7028,0,0,0,82,95.800003,...,NaN,NaN,1.173874,NaN,NaN,NaN,NaN,NaN,NaN,0


In [48]:
# There are 46 different areas observed (i)
crime2['ids'] = crime2.area

In [49]:
crime2.groupby('ids').diff().dropna()

,pop,crimes,unem,officers,pcinc,west,nrtheast,south,year,area,...,clpop,clcrmrte,lpolpc,clpolpc,cllawexp,cunem,clpopden,lcrmrt_1,ccrmrte,t


In [53]:
crime2["crmrte"]

0     74.657562
1     70.117294
2     92.934868
3     89.972214
4     83.611130
        ...    
87    74.098511
88    54.063602
89    54.983273
90    71.061371
91    82.907127
Name: crmrte, Length: 92, dtype: float64

In [54]:
crime2.groupby("ids")["crmrte"].diff()

0           NaN
1     -4.540268
2           NaN
3     -2.962654
4           NaN
        ...    
87    -3.127060
88          NaN
89     0.919670
90          NaN
91    11.845757
Name: crmrte, Length: 92, dtype: float64

In [50]:
crime2["crmrte_diff"] = crime2.groupby("ids")["crmrte"].diff()
crime2["unem_diff"] = crime2.groupby("ids")["unem"].diff()

In [60]:
crime2[['ids', 't', 'crmrte', 'unem']]

,ids,t,crmrte,unem
0,44.599998,0,74.657562,8.2
1,44.599998,1,70.117294,3.7
2,375.000000,0,92.934868,8.1
3,375.000000,1,89.972214,5.4
4,49.799999,0,83.611130,9.0
...,...,...,...,...
87,53.000000,1,74.098511,6.3
88,255.899994,0,54.063602,5.5
89,255.899994,1,54.983273,6.3
90,95.800003,0,71.061371,13.1


In [61]:
crime2[['ids', 't', 'crmrte', 'unem']].groupby('ids').diff().dropna()

,t,crmrte,unem
1,1.0,-4.540268,-4.500000
3,1.0,-2.962654,-2.700000
5,1.0,-6.416374,-3.100000
7,1.0,-4.901543,-6.900001
9,1.0,-4.608994,-5.200000
11,1.0,-24.800095,-8.200000
13,1.0,13.450974,-3.900000
15,1.0,-21.323151,-3.300000
17,1.0,-28.448345,-3.800000
19,1.0,-8.132919,-5.600000


In [51]:
crime2

,pop,crimes,unem,officers,pcinc,west,nrtheast,south,year,area,...,clpolpc,cllawexp,cunem,clpopden,lcrmrt_1,ccrmrte,t,ids,crmrte_diff,unem_diff
0,229528.0,17136.0,8.2,326,8532,1,0,0,82,44.599998,...,NaN,NaN,NaN,NaN,NaN,NaN,0,44.599998,NaN,NaN
1,246815.0,17306.0,3.7,321,12155,1,0,0,87,44.599998,...,-0.088070,0.977952,-4.5,0.072615,4.312912,-4.540268,1,44.599998,-4.540268,-4.5
2,814054.0,75654.0,8.1,1621,7551,1,0,0,82,375.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,0,375.000000,NaN,NaN
3,933177.0,83960.0,5.4,1803,11363,1,0,0,87,375.000000,...,-0.030160,0.200762,-2.7,0.136568,4.531899,-2.962654,1,375.000000,-2.962654,-2.7
4,374974.0,31352.0,9.0,633,8343,1,0,0,82,49.799999,...,NaN,NaN,NaN,NaN,NaN,NaN,0,49.799999,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,280370.0,20775.0,6.3,623,9340,0,0,1,87,53.000000,...,0.020713,0.147894,-0.7,0.033710,4.346731,-3.127060,1,53.000000,-3.127060,-0.7
88,268887.0,14537.0,5.5,400,7704,0,0,1,82,255.899994,...,NaN,NaN,NaN,NaN,NaN,NaN,0,255.899994,NaN,NaN
89,340158.0,18703.0,6.3,504,12039,0,0,1,87,255.899994,...,-0.004007,0.107985,0.8,0.235119,3.990161,0.919670,1,255.899994,0.919670,0.8
90,645231.0,45851.0,13.1,2087,7028,0,0,0,82,95.800003,...,NaN,NaN,NaN,NaN,NaN,NaN,0,95.800003,NaN,NaN


In [52]:
crime2[["t","crmrte_diff", "unem_diff"]].dropna().head()

,t,crmrte_diff,unem_diff
1,1,-4.540268,-4.500000
3,1,-2.962654,-2.700000
5,1,-6.416374,-3.100000
7,1,-4.901543,-6.900001
9,1,-4.608994,-5.200000


In [31]:
model1 = smf.ols("crmrte ~ t + unem", crime2).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 crmrte   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.5501
Date:                Thu, 30 Nov 2023   Prob (F-statistic):              0.579
Time:                        16:38:43   Log-Likelihood:                -441.90
No. Observations:                  92   AIC:                             889.8
Df Residuals:                      89   BIC:                             897.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     93.4202     12.739      7.333      0.000      68.107     118.733
t              7.9404      7.975      0.996      0.322      -7.906      23.787
unem           0.4265      1.188      0.359      0.720      -1.935       2.788
==============================================================================
Omnibus:                        8.350   Durbin-Watson:                   1.157
Prob(Omnibus):                  0.015   Jarque-Bera (JB):                8.771
Skew:                           0.756   Prob(JB):                       0.0125
Kurtosis:                       2.935   Cond. No.                         40.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
fdiff = smf.ols("crmrte_diff ~ unem_diff", crime2).fit()
fdiff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            crmrte_diff   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     6.384
Date:                Thu, 30 Nov 2023   Prob (F-statistic):             0.0152
Time:                        16:38:46   Log-Likelihood:                -202.17
No. Observations:                  46   AIC:                             408.3
Df Residuals:                      44   BIC:                             412.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.4022      4.702      3.276      0.002       5.926      24.879
unem_diff      2.2180      0.878      2.527      0.015       0.449       3.987
==============================================================================
Omnibus:                        2.636   Durbin-Watson:                   1.146
Prob(Omnibus):                  0.268   Jarque-Bera (JB):                2.255
Skew:                           0.539   Prob(JB):                        0.324
Kurtosis:                       2.883   Cond. No.                         8.70
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [104]:
import linearmodels as plm

In [105]:
crime2 = crime2.set_index(['ids', 'year'])

In [106]:
plm.FirstDifferenceOLS.from_formula(formula = 'crmrte ~ tunem', data = crime2).fit()

C:\Users\kunzn\anaconda3\lib\site-packages\linearmodels\shared\utility.py:187: FutureWarning: In a future version of pandas all arguments of MultiIndex.set_levels except for the argument 'levels' will be keyword-only
  df.index = df.index.set_levels(final_levels, [0, 1])


Dep. Variable:,crmrte,R-squared:,1.964e-05
Estimator:,FirstDifferenceOLS,R-squared (Between):,-0.0027
No. Observations:,46,R-squared (Within):,1.964e-05
Date:,"Thu, Dec 01 2022",R-squared (Overall):,-0.0026
Time:,15:00:36,Log-likelihood,-207.19
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0009
Entities:,46,P-value,0.9764
Avg Obs:,2.0000,Distribution:,"F(1,45)"
Min Obs:,2.0000,,
Max Obs:,2.0000,F-statistic (robust):,0.0009


## Within Estimator

In [107]:
crime2 = crime2.reset_index()

In [108]:
Y = "crmrte"
X = ["unem", "t"]

mean_data = crime2.groupby("ids")[X+[Y]].mean()
mean_data.head()

,unem,t,crmrte
ids,,,
13.000000,11.30,0.5,68.506205
17.799999,5.75,0.5,166.898941
18.900000,7.60,0.5,108.058220
20.799999,4.95,0.5,142.893265
21.900000,6.55,0.5,71.647846


In [109]:
demeaned_data = (crime2.set_index("ids")[X+[Y]] - mean_data) # subtract the mean data
demeaned_data.head()

,unem,t,crmrte
ids,,,
13.000000,3.60,-0.5,4.807211
13.000000,-3.60,0.5,-4.807211
17.799999,3.35,-0.5,2.416580
17.799999,-3.35,0.5,-2.416580
18.900000,3.70,-0.5,-11.970970


In [110]:
crime2["crmrte_demmean"] = crime2.crmrte - crime2.groupby("ids", axis=0).transform('mean')["crmrte"]
crime2["unem_demmean"] = crime2.unem - crime2.groupby("ids", axis=0).transform('mean')["unem"]

In [111]:
mod = smf.ols('crmrte ~ t+unem-1', data=demeaned_data).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 crmrte   R-squared (uncentered):                   0.196
Model:                            OLS   Adj. R-squared (uncentered):              0.178
Method:                 Least Squares   F-statistic:                              10.97
Date:                Thu, 01 Dec 2022   Prob (F-statistic):                    5.43e-05
Time:                        15:00:42   Log-Likelihood:                         -340.57
No. Observations:                  92   AIC:                                      685.1
Df Residuals:                      90   BIC:                                      690.2
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
t             15.4022      3.288      4.685      0.000       8.871      21.934
unem           2.2180      0.614      3.614      0.000       0.999       3.437
==============================================================================
Omnibus:                        0.005   Durbin-Watson:                   2.890
Prob(Omnibus):                  0.998   Jarque-Bera (JB):                0.053
Skew:                           0.000   Prob(JB):                        0.974
Kurtosis:                       2.883   Cond. No.                         8.70
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [116]:
crime2 = crime2.set_index(['ids', 'year'], drop = False)

In [117]:
# Drop absorbed drops any variables that do not change over time (ethnicity would be an example)
fe_fit = plm.PanelOLS.from_formula(formula = 'crmrte ~ t+ unem + EntityEffects',
                                   data = crime2, drop_absorbed = True).fit()
fe_fit

Dep. Variable:,crmrte,R-squared:,0.1961
Estimator:,PanelOLS,R-squared (Between):,0.4064
No. Observations:,92,R-squared (Within):,0.1961
Date:,"Thu, Dec 01 2022",R-squared (Overall):,0.4041
Time:,15:11:26,Log-likelihood,-340.57
Cov. Estimator:,Unadjusted,,
,,F-statistic:,5.3653
Entities:,46,P-value,0.0082
Avg Obs:,2.0000,Distribution:,"F(2,44)"
Min Obs:,2.0000,,
Max Obs:,2.0000,F-statistic (robust):,5.3653


### Dummy variable Regression

In [23]:
smf.ols(formula = 'crmrte ~ t+ unem + C(ids)', data = crime2).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 crmrte   R-squared:                       0.891
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     7.642
Date:                Thu, 01 Dec 2022   Prob (F-statistic):           1.70e-10
Time:                        11:44:37   Log-Likelihood:                -340.57
No. Observations:                  92   AIC:                             777.1
Df Residuals:                      44   BIC:                             898.2
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       35.7417     15.515      2.304      0.026       4.473      67.010
C(ids)[T.17.799999237060547]   110.7026     14.992      7.384      0.000      80.489     140.917
C(ids)[T.18.899999618530277]    47.7586     14.545      3.283      0.002      18.444      77.073
C(ids)[T.20.799999237060547]    88.4714     15.235      5.807      0.000      57.768     119.175
C(ids)[T.21.899999618530277]    13.6771     14.779      0.925      0.360     -16.107      43.461
C(ids)[T.24.100000381469727]    51.7858     14.231      3.639      0.001      23.105      80.467
C(ids)[T.24.200000762939453]    24.2242     15.006      1.614      0.114      -6.019      54.467
C(ids)[T.25.299999237060547]    48.2313     14.978      3.220      0.002      18.046      78.417
C(ids)[T.27.399999618530277]    30.3498     14.791      2.052      0.046       0.540      60.159
C(ids)[T.34.20000076293945]     43.7282     14.545      3.006      0.004      14.414      73.042
C(ids)[T.34.29999923706055]     85.1854     14.315      5.951      0.000      56.335     114.035
C(ids)[T.41.79999923706055]      7.3826     14.205      0.520      0.606     -21.246      36.011
C(ids)[T.44.59999847412109]     15.7475     14.936      1.054      0.297     -14.353      45.848
C(ids)[T.46.400001525878906]    27.7747     14.730      1.886      0.066      -1.912      57.461
C(ids)[T.49.79999923706055]     20.4360     14.575      1.402      0.168      -8.939      49.811
C(ids)[T.52.900001525878906]    28.3713     14.950      1.898      0.064      -1.758      58.500
C(ids)[T.53.0]                  17.4695     14.754      1.184      0.243     -12.265      47.204
C(ids)[T.54.400001525878906]    49.5983     14.181      3.497      0.001      21.018      78.179
C(ids)[T.55.09999847412109]     56.3623     14.992      3.760      0.000      26.148      86.576
C(ids)[T.55.29999923706055]     59.3131     14.239      4.166      0.000      30.616      88.010
C(ids)[T.55.400001525878906]    11.4640     14.327      0.800      0.428     -17.411      40.339
C(ids)[T.55.900001525878906]    34.2429     14.779      2.317      0.025       4.459      64.027
C(ids)[T.61.09999847412109]      8.9101     14.354      0.621      0.538     -20.018      37.838
C(ids)[T.74.0]                  22.7542     14.303      1.591      0.119      -6.072      51.580
C(ids)[T.78.0]                  15.8335     14.261      1.110      0.273     -12.908      44.575
C(ids)[T.79.0]                  20.3167     14.179      1.433      0.159      -8.260      48.893
C(ids)[T.95.8000030517578]      12.4704     14.266      0.874      0.387     -16.280      41.221
C(ids)[T.97.3000030517578]      40.2455     14.224      2.829      0.007      11.579      68.912
C(ids)[T.100.5]                 60.1289     14.606      4.117      0.000    

## Random Effects

Random effects assumes that the unobserved heterogeneity $a_i$ are uncorrelated with the regressors. If this is true, then we could simply run ols on our data as if it was a cross section. 

However the errors in our model will be serially correlated across time, which means the standard errors in our model will be incorrect.

Consider the model from earlier:

$$crmrte_{it} = \beta_0 + \beta_1 unem_{it} + a_i + u_{it}$$

where $v_{it}= a_i + u_{it}$ is the composite error term. If our random effects assumptions hold, then:
$cov(x_{itj}, a_i) = 0$  for all $t = 0, 1, ..., T$ and $j = 1,2,...,k$.
 
Unfortunately, one of our assumptions to derive correct standard errors is that there is no serial correlation across time. This can't be true, since $a_i$ is included in the composite error term. In fact, under the ranedom effects assumptions:

$$cov(v_{it}, v_{is}) = \frac{\sigma^2_a}{\sigma^2_a+\sigma^2_u}$$ $$t \neq s$$

GLS can be used to solve the serial correlation problem.

In [43]:
reg_re = plm.RandomEffects.from_formula(formula = 'crmrte ~ 1+ t  + unem', data = crime2).fit()
reg_re

Dep. Variable:,crmrte,R-squared:,0.0927
Estimator:,RandomEffects,R-squared (Between):,-0.0320
No. Observations:,92,R-squared (Within):,0.1911
Date:,"Thu, Dec 01 2022",R-squared (Overall):,-0.0017
Time:,11:56:44,Log-likelihood,-372.87
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.5472
Entities:,46,P-value,0.0132
Avg Obs:,2.0000,Distribution:,"F(2,89)"
Min Obs:,2.0000,,
Max Obs:,2.0000,F-statistic (robust):,4.5472


## Hauseman Test For Endogeneity

The random effects estimates aere reliable under the assumption that individual characteristics are exogenous (I.e. they are independent wrt to the resgressors). We have already used the Hausman test duruing the IV lecture to check for endogeneity, and it can be applied in this context as well. 

The null hypothesis of the test is that individual random effects are exogenous. The test compares the random and fixed effects models.

In [45]:
ch3 = pd.read_csv("chemical3.csv", parse_dates = True)
ch3 = ch3.set_index(["firm", "year"])

In [47]:
reg_re = plm.RandomEffects.from_formula(formula = 'lsales ~ 1+ lcapital + llabor', data = ch3).fit()
fe_fit = plm.PanelOLS.from_formula(formula = 'lsales ~ lcapital + llabor + EntityEffects', data = ch3, drop_absorbed = True).fit()


In [53]:
reg_re = plm.RandomEffects.from_formula(formula = 'crmrte ~ 1+ t  + unem', data = crime2).fit()
fe_fit = plm.PanelOLS.from_formula(formula = 'crmrte ~ t+ unem + EntityEffects',
                                   data = crime2, drop_absorbed = True).fit()

(98.81660543073161, 2, 3.485353000599886e-22)

In [52]:
def PnlHausman(fe_fit, re_fit):
    # pull out the variances and parameters for test
    Dcov = fe_fit.cov - re_fit.cov.iloc[1:, 1:]
    dparams = fe_fit.params - re_fit.params[1:]
    # get the test statistic
    Chi2 = dparams.dot(np.linalg.inv(Dcov)).dot(dparams)
    # calculate the degrees of freedom
    dof = re_fit.params.size - 1
    # calculate the p-value
    pvalue = stats.chi2(dof).sf(Chi2)
    return(Chi2, dof, pvalue)
    

In [115]:
# We fail to reject the null hypothesis of exogeneity
# Random effects may be appropriate
PnlHausman(fe_fit, reg_re)

(1.7882988651011655, 2, 0.40895529481239756)